Before running the notebook, open a terminal and run the following commands:

1. Build the container image:
    ```bash
    docker build -t whisper-x-api .
    ```

2. Ensure the container whisper-x-test-container is stopped:
    ```bash
    docker stop whisper-x-test-container
    ```

3. Ensure the container whisper-x-test-container is removed:
    ```bash
    docker rm whisper-x-test-container
    ```

4. Start the container:
    ```bash
    docker run -d -p 8000:8000 --gpus all -v ${HOME}/.config/gcloud:/root/.config/gcloud --name whisper-x-test-container whisper-x-api
    ```

In [1]:
import time
import requests
import json

def call_local_whisperx(uri: str):
    # Endpoint FastAPI locale
    url = "http://localhost:8000/predict"

    # Dati da inviare (come nel curl)
    payload = {
        "instances": [
            uri
        ],
        "parameters": {}
    }

    # Headers
    headers = {
        "Content-Type": "application/json"
    }

    # Invio richiesta POST
    response = requests.post(url, headers=headers, data=json.dumps(payload))

    # Output del risultato
    if response.ok:
        return response.json()   
    else:
        return "Errore!!!"

In [ ]:
from utils.names import get_file_names

files = get_file_names()

In [ ]:
for file in files:
    print(f"Starting prediction of file {file}")
    
    uri = f"gs://my-bucket/audio/{file}.wav"

    start = time.time()
    response = call_local_whisperx(uri)
    end = time.time()
    
    print(f"Saving prediction of file {file}")
    
    filename = file.replace('.wav','')

    with open(f"predictions/{filename}.json", "w", encoding="utf-8") as f:
        json.dump(response, f, indent=4, ensure_ascii=False)
        
    print(f"Saving prediction time of file {file}: {end-start} seconds")
        
    with open("times.txt", "a", encoding="utf-8") as f:
        f.write(f"{file} : {end - start}\n")